In [1]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torchvision import models, datasets
from torch import optim
import torch.optim as optim
import torch.nn.functional as F
from torchvision.models import resnet18, vgg16, alexnet
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report


In [ ]:
# RESNET18

In [22]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torchvision import models, datasets
from torch import optim
import torch.optim as optim
import torch.nn.functional as F
from torchvision.models import resnet18, vgg16, alexnet
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report



class FER2013Dataset(Dataset):
    def __init__(self, root_dir, transform = None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.label_to_idx = {}  # Dictionary to map labels to indices

         # Traverse through all the folders and subfolders to collect image paths and labels
        for idx, label in enumerate(sorted(os.listdir(root_dir))):  # Sort labels to ensure consistent ordering
            label_dir = os.path.join(root_dir, label)
            if os.path.isdir(label_dir):
                self.label_to_idx[label] = idx  # Map each label to an integer index
                for img_name in os.listdir(label_dir):
                    img_path = os.path.join(label_dir, img_name)
                    self.image_paths.append(img_path)
                    self.labels.append(idx)  # Use index as label

    
    def __len__(self):
        #total no of samples
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        img = Image.open(img_path).convert('RGB')

        if self.transform:
            img = self.transform(img)
        
        return img,label

# Define the transformations to be applied to the images   
transform = transforms.Compose([
    
    transforms.Resize((224,224)),  # Resize image to match pre-trained model input
    transforms.ToTensor(),  # Convert PIL image to PyTorch tensor
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229,0.224, 0.225])  #Normalize
])


# Create an instance of the dataset
train_dataset = FER2013Dataset(root_dir=r'C:\Users\USER\Desktop\Final_Projects\.venv\Emotion_Detection\FER2013FolderDataset\train', transform=transform)
test_dataset = FER2013Dataset(root_dir=r'C:\Users\USER\Desktop\Final_Projects\.venv\Emotion_Detection\FER2013FolderDataset\test', transform=transform)

# Create DataLoaders for training and testing
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


model = models.resnet18(pretrained=True)


# Freeze all layers except the final layer
for param in model.parameters():
    param.requires_grad = False

# Modify the final layer to output 7 classes (for FER2013)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 7)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


for epoch in range(5):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch} loss: {running_loss / len(train_loader)}')





# Evaluate on the test set
model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        # Collect all true labels and predictions
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Convert lists to numpy arrays
all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

# Calculate precision, recall, and F1 score
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

# Print classification report
report = classification_report(all_labels, all_preds, target_names=[str(i) for i in range(7)])

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print('Classification Report:')
print(report)

# Calculate and print accuracy
correct = (all_preds == all_labels).sum()
total = len(all_labels)
accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy:.2f}%')


# Optional: Save the model after training
torch.save(model.state_dict(), 'fer2013_resnet18.pth')


c:\Users\USER\Desktop\Final_Projects\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\USER\Desktop\Final_Projects\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0 loss: 1.6100254121894029
Epoch 1 loss: 1.5042699554184231
Epoch 2 loss: 1.4819937258095943
Epoch 3 loss: 1.4680913475680193
Epoch 4 loss: 1.4572801490403495
Precision: 0.4229
Recall: 0.4330
F1 Score: 0.4198
Classification Report:
              precision    recall  f1-score   support

           0       0.29      0.43      0.35       958
           1       0.00      0.00      0.00       111
           2       0.30      0.23      0.26      1024
           3       0.56      0.63      0.59      1774
           4       0.43      0.39      0.41      1233
           5       0.41      0.25      0.31      1247
           6       0.49      0.67      0.57       831

    accuracy                           0.43      7178
   macro avg       0.36      0.37      0.36      7178
weighted avg       0.42      0.43      0.42      7178

Accuracy on the test set: 43.30%


In [ ]:
# VGG16

In [3]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torchvision import models, datasets
from torch import optim
import torch.optim as optim
import torch.nn.functional as F
from torchvision.models import resnet18, vgg16, alexnet
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report



class FER2013Dataset(Dataset):
    def __init__(self, root_dir, transform = None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.label_to_idx = {}  # Dictionary to map labels to indices

         # Traverse through all the folders and subfolders to collect image paths and labels
        for idx, label in enumerate(sorted(os.listdir(root_dir))):  # Sort labels to ensure consistent ordering
            label_dir = os.path.join(root_dir, label)
            if os.path.isdir(label_dir):
                self.label_to_idx[label] = idx  # Map each label to an integer index
                for img_name in os.listdir(label_dir):
                    img_path = os.path.join(label_dir, img_name)
                    self.image_paths.append(img_path)
                    self.labels.append(idx)  # Use index as label

    
    def __len__(self):
        #total no of samples
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        img = Image.open(img_path).convert('RGB')

        if self.transform:
            img = self.transform(img)
        
        return img,label

# Define the transformations to be applied to the images   
transform = transforms.Compose([
    
    transforms.Resize((224,224)),  # Resize image to match pre-trained model input
    transforms.ToTensor(),  # Convert PIL image to PyTorch tensor
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229,0.224, 0.225])  #Normalize
])


# Create an instance of the dataset
train_dataset = FER2013Dataset(root_dir=r'C:\Users\USER\Desktop\Final_Projects\.venv\Emotion_Detection\FER2013FolderDataset\train', transform=transform)
test_dataset = FER2013Dataset(root_dir=r'C:\Users\USER\Desktop\Final_Projects\.venv\Emotion_Detection\FER2013FolderDataset\test', transform=transform)

# Create DataLoaders for training and testing
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


model = models.vgg16(pretrained=True)


# Freeze all layers except the final layer
for param in model.parameters():
    param.requires_grad = False


# Modify the classifier for 7 classes (FER2013)
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 7)

# Modify the final layer to output 7 classes (for FER2013)
#num_features = model.fc.in_features
#model.fc = nn.Linear(num_features, 7)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


for epoch in range(5):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch} loss: {running_loss / len(train_loader)}')

# Evaluate on the test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
print(f'Accuracy on the test set: {100 * correct / total}%')


# Evaluate on the test set
model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        # Collect all true labels and predictions
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Convert lists to numpy arrays
all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

# Calculate precision, recall, and F1 score
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

# Print classification report
report = classification_report(all_labels, all_preds, target_names=[str(i) for i in range(7)])

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print('Classification Report:')
print(report)

# Calculate and print accuracy
correct = (all_preds == all_labels).sum()
total = len(all_labels)
accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy:.2f}%')


# Optional: Save the model after training
torch.save(model.state_dict(), 'fer2013_VGG16.pth')

c:\Users\USER\Desktop\Final_Projects\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\USER\Desktop\Final_Projects\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0 loss: 1.7199949007522821
Epoch 1 loss: 1.6844026460944943
Epoch 2 loss: 1.6770031515898842
Epoch 3 loss: 1.6625328877057157
Epoch 4 loss: 1.6660226578170845
Accuracy on the test set: 40.860964056840345%


c:\Users\USER\Desktop\Final_Projects\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\USER\Desktop\Final_Projects\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\USER\Desktop\Final_Projects\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

Precision: 0.4327
Recall: 0.4086
F1 Score: 0.3734
Classification Report:
              precision    recall  f1-score   support

           0       0.36      0.24      0.28       958
           1       0.00      0.00      0.00       111
           2       0.34      0.19      0.24      1024
           3       0.44      0.74      0.55      1774
           4       0.32      0.58      0.41      1233
           5       0.51      0.11      0.19      1247
           6       0.73      0.42      0.54       831

    accuracy                           0.41      7178
   macro avg       0.38      0.33      0.32      7178
weighted avg       0.43      0.41      0.37      7178

Accuracy on the test set: 40.86%


In [ ]:
# Alexnet

In [2]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torchvision import models, datasets
from torch import optim
import torch.optim as optim
import torch.nn.functional as F
from torchvision.models import resnet18, vgg16, alexnet
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report


class FER2013Dataset(Dataset):
    def __init__(self, root_dir, transform = None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.label_to_idx = {}  # Dictionary to map labels to indices

         # Traverse through all the folders and subfolders to collect image paths and labels
        for idx, label in enumerate(sorted(os.listdir(root_dir))):  # Sort labels to ensure consistent ordering
            label_dir = os.path.join(root_dir, label)
            if os.path.isdir(label_dir):
                self.label_to_idx[label] = idx  # Map each label to an integer index
                for img_name in os.listdir(label_dir):
                    img_path = os.path.join(label_dir, img_name)
                    self.image_paths.append(img_path)
                    self.labels.append(idx)  # Use index as label

    
    def __len__(self):
        #total no of samples
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        img = Image.open(img_path).convert('RGB')

        if self.transform:
            img = self.transform(img)
        
        return img,label

# Define the transformations to be applied to the images   
transform = transforms.Compose([
    
    transforms.Resize((224,224)),  # Resize image to match pre-trained model input
    transforms.ToTensor(),  # Convert PIL image to PyTorch tensor
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229,0.224, 0.225])  #Normalize
])


# Create an instance of the dataset
train_dataset = FER2013Dataset(root_dir=r'C:\Users\USER\Desktop\Final_Projects\.venv\Emotion_Detection\FER2013FolderDataset\train', transform=transform)
test_dataset = FER2013Dataset(root_dir=r'C:\Users\USER\Desktop\Final_Projects\.venv\Emotion_Detection\FER2013FolderDataset\test', transform=transform)

# Create DataLoaders for training and testing
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


model = models.alexnet(pretrained=True)


# Freeze all layers except the final layer
for param in model.parameters():
    param.requires_grad = False


# Modify the classifier for 7 classes (FER2013)
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 7)

# Modify the final layer to output 7 classes (for FER2013)
#num_features = model.fc.in_features
#model.fc = nn.Linear(num_features, 7)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


for epoch in range(5):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch} loss: {running_loss / len(train_loader)}')

# Evaluate on the test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
print(f'Accuracy on the test set: {100 * correct / total}%')

# Evaluate on the test set
model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        # Collect all true labels and predictions
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Convert lists to numpy arrays
all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

# Calculate precision, recall, and F1 score
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

# Print classification report
report = classification_report(all_labels, all_preds, target_names=[str(i) for i in range(7)])

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print('Classification Report:')
print(report)

# Calculate and print accuracy
correct = (all_preds == all_labels).sum()
total = len(all_labels)
accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy:.2f}%')


# Optional: Save the model after training
torch.save(model.state_dict(), 'fer2013_alexnet.pth')

c:\Users\USER\Desktop\Final_Projects\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0 loss: 1.75423906295496
Epoch 1 loss: 1.7201677021842225
Epoch 2 loss: 1.7166682953027415
Epoch 3 loss: 1.689706286369825
Epoch 4 loss: 1.6937163422394437
Accuracy on the test set: 37.921426581220395%
Precision: 0.4823
Recall: 0.3792
F1 Score: 0.3799
Classification Report:
              precision    recall  f1-score   support

           0       0.27      0.41      0.32       958
           1       0.31      0.24      0.27       111
           2       0.22      0.60      0.32      1024
           3       0.70      0.41      0.52      1774
           4       0.43      0.29      0.35      1233
           5       0.53      0.10      0.17      1247
           6       0.62      0.57      0.59       831

    accuracy                           0.38      7178
   macro avg       0.44      0.37      0.36      7178
weighted avg       0.48      0.38      0.38      7178

Accuracy on the test set: 37.92%
